# Evaluation of Blocking

For each entity we select the largest cluster (block) for that entity. True positives are items that are in the largest cluster and are correctly associated with that entity, false positives are items in the cluster that are not related to the entity and false negatives are items that should be in the cluster but aren't. 

We indicate true positives with $TP$, false positives with $FP$ and false negatives with $FN$,
then we have the equations for $precision$, $recall$ and $f{\text -}measure$:

$$ precision = \frac{TP}{TP + FP} \\ $$
$$ recall = \frac{TP}{TP + FN} \\ $$
$$ f{\text -}measure = \frac{2 \cdot precision \cdot recall}{precision + recall} = \frac{2 \cdot TP}{2 TP + FP + FN} \\ $$


In [6]:
import paths
import json
import evaluation

In [7]:
algorithm = "hdbscan"

evaluation.generate_entity2clusters(algorithm)

with open(paths.RESULTS_DIR + "/evaluation/" + algorithm + "_entity2clusters.json","r") as file:
    entity2clusters: dict[str, dict[str, list[str]]] = json.load(file)
    
with open(paths.RESULTS_DIR + "/clustering/" + algorithm + "/" + algorithm + "_clusters.json", "r") as file:
    alg_clusters = json.load(file)

In [8]:
TP = 0
FP = 0
FN = 0

In [9]:
# Remove clusters with id -1 and count the number of false negatives items in those clusters
for clusters in entity2clusters.values():
    for cluster_id, values in clusters.items():
        if cluster_id == '-1':
            FN += len(values)
            del clusters[cluster_id]
            break

In [10]:
for entity, clusters in entity2clusters.items():
    if len(clusters) == 0:
        continue
    # Get the cluster with the most elements
    max_cluster = max(clusters.values(), key=len)
    
    # Get the id of the cluster with the most elements
    for key,value in clusters.items():
        if len(value) == len(max_cluster):
             max_cluster_id = key
             
    # Calculate False Positives (FP) as elements in the largest cluster that should not be there
    for item in alg_clusters[max_cluster_id]:
        if item not in max_cluster:
            FP += 1   
                                 
    # Calculate True Positives (TP) as elements in the largest cluster
    TP += len(max_cluster)
    
    # Calculate False Negatives (FN) as elements in the other clusters
    for cluster in clusters.values():
        if cluster != max_cluster:
            FN += len(cluster)
     
precision = TP / (TP + FP) if (TP + FP) > 0 else 0
recall = TP / (TP + FN) if (TP + FN) > 0 else 0
f_measure = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F-measure: {f_measure}")

Precision: 0.0019691966869804177
Recall: 0.8970652650021901
F-measure: 0.003929766929130837
